# Shadow Detection and Removal

## Introduction

A shadow appears on an area when the light from a source cannot reach the area due to obstruction by an object.

Shadow in general are of two types: hard and soft shadows. The soft shadows retain the texture of the background surface, whereas the hard shadows are too dark and have little texture.

Most of the shadow detection methods need multiple images for camera calibration, But there are some techniques that able to extract shadows from a single image where we will consider one of them for our project.

A shadow detection method is selected based on the mean value of RGB image in A and B planes of LAB equivalent of the image.


### The LAB color space


<img align="right" src="img/lab_color_space.jpg" style=" width:300px; padding: 0 20px;  " />

The LAB color space has three channels − L is the Lightness channel, A and B are the two color channels.

The L channel has values ranging from 0 up to 100, which correspond to different shades from black to white. The A channel has values ranging from −128 up to +127 and gives the red to green ratio. The B channel also has values ranging from −128 up to +127 and gives the yellow to blue ratio.

Thus, a high value in A or B channel represents a color having more red or yellow and a low value represents a color having more green or blue.

We will apply this task by Detecting the shadows of objects first, then try to remove these shadows.


### 1. Shadow Detection


An approach to detect the shadows areas in a single RGB image is convert from RGB to LAB color space. Since the shadow regions are darker and less illuminated than the surroundings, it is easy to locate them in the L channel since the L channel gives lightness information. The B channel values are also lesser in the shadow areas in most of the outdoor images.

Thus combining the values from L and B channels, the pixels with values less than a threshold are identified as shadow pixels, and others as non-shadow pixels. The method works well only for images whose yellow to blue ratio is maintained within a range.

The mean value of the image in A and B channels are calculated.

The major steps involved in the shadow detection phase are:

1. Convert the RGB image to a LAB image.

2. Compute the mean values of the pixels in L, A and B planes of the image separately.

3. If mean (A) + mean (B) ≤ 256 then Classify the pixels with a value in L ≤(mean(L) – standard deviation (L)/3) as shadow pixels and others as non-shadow pixels.

4. Else classify the pixels with lower values in both L and B planes as shadow pixels and others as non-shadow pixels.

The shadow detection using this pixel-based method may classify some non shadow pixels as shadow pixels. Isolated pixels are removed using morphological
operation called cleaning.

The misclassified pixels are removed using dilation followed by erosion. Also area-based thresholding is done, so that only regions with a number of pixels greater than a threshold can be considered as shadow regions. All these morphological operations will help to eliminate misclassification of pixels.


### 2. Shadow Removal and Edge Correction


Shadow removal is done by multiplying R, G and B channels of the shadow pixels using appropriate constants. Each shadow region is considered separately. The ratio of the average of each channel in the near non-shadow region to that in the shadow region is taken as a constant for each channel. The shadow regions achieve almost the same illumination as the non-shadow regions. But over-illumination may occur towards the edges of shadow.

Since shadow regions are not uniformly illuminated, the same constant for the entire shadow region will create over-illuminated areas near the shadow edges. This is overcome by applying a median filter on the over-illuminated areas. Thus a shadow-free image without over-illuminated edges is obtained.


## Implementation

In [ ]:
from expt_utils import *

In [ ]:
def find_shadow_mask(img_rgb, cln_kernel=np.ones((3, 3), np.uint8)):
    """
    ...
    """
    img_lab = cv.cvtColor(img_rgb, cv.COLOR_RGB2LAB)

    mean_l = np.mean(img_lab[:, :, 0])
    mean_a = np.mean(img_lab[:, :, 1])
    mean_b = np.mean(img_lab[:, :, 2])
    std_l = np.std(img_lab[:, :, 0])

    shadow_mask = (
        (img_lab[:, :, 0] <= (mean_l - std_l / 3))
        if mean_a + mean_b <= 256 else
        ((img_lab[:, :, 0] < mean_l) & (img_lab[:, :, 2] < mean_b))
    ).astype(np.uint8) * 255

    shadow_mask_cln = cv.morphologyEx(shadow_mask, cv.MORPH_OPEN, cln_kernel)
    shadow_mask_cln = cv.dilate(shadow_mask_cln, cln_kernel, iterations=1)
    shadow_mask_cln = cv.erode(shadow_mask_cln, cln_kernel, iterations=1)

    return shadow_mask_cln

In [ ]:
def shadow_remove(img, shadow_mask):
    """
    ...
    """
    img_float = img.astype(np.float32)

    in_shadow_pixels = np.where(shadow_mask)
    out_shadow_pixels = np.where(~shadow_mask)

    avg_rgb_in_shadow = np.array([
        np.mean(img_float[*in_shadow_pixels, c])
        for c in range(3)
    ])
    avg_rgb_out_shadow = np.array([
        np.mean(img_float[*out_shadow_pixels, c])
        for c in range(3)
    ])
    constants = avg_rgb_out_shadow / avg_rgb_in_shadow

    for c in range(3):
        img_float[*in_shadow_pixels, c] *= constants[c]

    img_shadow_removed = np.clip(img_float, 0, 255).astype(np.uint8)
    return img_shadow_removed

In [ ]:
img_rgb = plt.imread(f'{DS_DIR}/frames/train/00049/00049_3880.jpg')
img_lab = cv.cvtColor(img_rgb, cv.COLOR_RGB2LAB)

img_l, img_a, img_b = cv.split(img_lab)

hist_l = cv.calcHist([img_lab], [0], None, [256], [0, 256]).flatten()
hist_a = cv.calcHist([img_lab], [1], None, [256], [0, 256]).flatten()
hist_b = cv.calcHist([img_lab], [2], None, [256], [0, 256]).flatten()

plt.figure(figsize=(10, 6), tight_layout=True)

plt.subplot(3, 3, (1, 3)), plt.axis('off'), plt.title('Image')
plt.imshow(img_rgb)


plt.subplot(3, 3, 4), plt.axis('off'), plt.title('L Channel')
plt.imshow(img_l, cmap='gray')

plt.subplot(3, 3, 5), plt.axis('off'), plt.title('A Channel')
plt.imshow(img_a, cmap='gray')

plt.subplot(3, 3, 6), plt.axis('off'), plt.title('B Channel')
plt.imshow(img_b, cmap='gray')

plt.subplot(3, 3, 7), plt.title('L Channel Histogram')
plt.bar(range(256), hist_l, color='k')

plt.subplot(3, 3, 8), plt.title('A Channel Histogram')
plt.bar(range(256), hist_a, color='k')

plt.subplot(3, 3, 9), plt.title('B Channel Histogram')
plt.bar(range(256), hist_b, color='k')

plt.savefig(f'{OUT_DIR}/03-01-img_lab_hist')

In [ ]:
img_shadow_mask = find_shadow_mask(img_rgb)
img_shadow_removed = shadow_remove(img_rgb, img_shadow_mask)

plt.figure(figsize=(10, 2.3), tight_layout=True)

plt.subplot(1, 3, 1), plt.axis('off'), plt.title('Image')
plt.imshow(img_rgb)

plt.subplot(1, 3, 2), plt.axis('off'), plt.title('Shadow Mask')
plt.imshow(img_shadow_mask, cmap='gray')

plt.subplot(1, 3, 3), plt.axis('off'), plt.title('Image Shadow Removed')
plt.imshow(img_shadow_removed)

plt.savefig(f'{OUT_DIR}/03-01-detect_and_remove_shadow')

## Conclusion

We found that shadow edge correction is done to reduce the errors in the shadow boundary very well.

We only could reduce the errors in the shadow and not to remove it at all because several factors, for examples:
- Extract shadows only from a single image: this lead the algorithm to Limited Perspective, Ambiguity in Shadow Detection and Loss of Depth Information.
- Non-uniform illumination: The illumination is not uniform in the shadow region. Towards the shadow boundary, diffusion takes place.